In [1]:
from hcipy import *
import numpy as np
from matplotlib import pyplot as plt

N = 64
D = 9.96
aperture = circular_aperture(D)
pupil_grid = make_pupil_grid(N, D)
sps = 40 * N // 128
pupsep = 1
outgrid_size = int(np.ceil(sps * (pupsep + 1)))
D_grid = 3.6e-3
pyramid_grid = make_pupil_grid(N, D_grid)
outgrid = np.zeros((N, N)).tolist()
buffer = (N - 2 * outgrid_size)//2

def aberration_basis(N):
    wfref = Wavefront(aperture(pupil_grid))
    wfref.electric_field.shape = (N, N)
    aberration_mode_basis = []
    for i in range(N):
        for j in range(N):
            l = np.zeros((N, N), dtype=complex)
            l[i][j] = 1j
            aberration_mode_basis.append(Wavefront(Field(np.round(np.asarray(l).ravel(), 3) * aperture(pupil_grid), wfref.grid)))
    return aberration_mode_basis

basis = aberration_basis(N)
print(len(basis))

4096


In [6]:
def get_sub_images(electric_field):
    pyramid_grid = make_pupil_grid(N, D_grid)
    images = Field(np.asarray(electric_field).ravel(), pyramid_grid)
    pysize = int(np.sqrt(images.size))
    images.shape = (pysize, pysize)
    sub_images = [images[pysize-sps-1:pysize-1, 0:sps], images[pysize-sps-1:pysize-1, pysize-sps-1:pysize-1],
                  images[0:sps, 0:sps], images[0:sps, pysize-sps-1:pysize-1]]
    subimage_grid = make_pupil_grid(sps, D_grid * sps / N)
    for count, img in enumerate(sub_images):
        img = img.ravel()
        img.grid = subimage_grid
        sub_images[count] = img
    return sub_images

def pyramid_prop(wf):
    # Given a wavefront, returns the result of a pyramid propagation and splitting into sub-images, 
    # as a list of hcipy Field objects.
    keck_pyramid = PyramidWavefrontSensorOptics(pupil_grid, pupil_separation=pupsep, num_pupil_pixels=sps)
    return get_sub_images(keck_pyramid.forward(wf).electric_field)

def estimate(images_list):
    EstimatorObject = PyramidWavefrontSensorEstimator(aperture, make_pupil_grid(sps*2, D_grid*sps*2/N))
    I_b = images_list[0]
    I_a = images_list[1]
    I_c = images_list[2]
    I_d = images_list[3]
    norm = I_a + I_b + I_c + I_d
    if all([el == 0 for el in norm.tolist()]):
        return
    I_x = (I_a + I_b - I_c - I_d) / norm
    I_y = (I_a - I_b - I_c + I_d) / norm
    pygrid = make_pupil_grid(sps)
    return Field(I_x.ravel(), pygrid), Field(I_y.ravel(), pygrid)

def make_slopes(wf):
    try:
        x, y = estimate(pyramid_prop(wf))
    except TypeError:
        return
    return np.concatenate((x, y))

In [7]:
slopes_basis = [make_slopes(basis_element) for basis_element in basis]

In [8]:
np.asarray(slopes_basis).shape

(4096,)

In [10]:
sum([(1 if el is None else 0) for el in slopes_basis])

1000

In [15]:
slopes_basis[sps//2 + 1000].shape

(800,)

In [18]:
for index, el in enumerate(slopes_basis):
    if el is None:
        slopes_basis[index] = np.zeros((2*sps*sps,))

In [ ]:
#This is just messy. I've just added 800k zeros to this and I don't think it's done a thing that'll help me.
reconstructor = lambda wf: 